In [1]:
import pandas as pd

In [2]:
week1 = pd.read_csv("data/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("data/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("data/Restaurant - Customers.csv")
foods = pd.read_csv("data/Restaurant - Foods.csv")


## concat method

In [5]:
# combine two dataframes into one
# ignores current index and generate a new index for each row
df = pd.concat([week1, week2], ignore_index=True)
df

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [6]:
# alternatively, create a multi-index dataframe and keep original index
df = pd.concat([week1, week2], keys=["Week1", "Week2"])
df

Customer ID  Food ID
Week1 0            537        9
      1             97        4
      2            658        1
      3            202        2
      4            155        9
...                ...      ...
Week2 245          783       10
      246          556       10
      247          547        9
      248          252        9
      249          249        6

[500 rows x 2 columns]

In [7]:
# get data for Week1
df.loc["Week1"]

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


In [8]:
# get row for multi-index position Week1/51
df.loc["Week1", 51]

Customer ID    682
Food ID         10
Name: (Week1, 51), dtype: int64

## append() method (on dataframe)

In [12]:
all_sales = week2.append(week1, ignore_index=True)
all_sales

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3
...,...,...
495,413,9
496,926,6
497,134,3
498,396,6


## merge() with inner joins

In [14]:
# on parameter only works if columns are identical in both dataframes
week1.merge(week2, how="inner", on="Customer ID")
# notice the values of x and y for duplicate names

,Customer ID,Food ID_x,Food ID_y
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


In [15]:
# use suffixes to 
week1.merge(week2, how="inner", on="Customer ID", suffixes=[" - WK1", " - WK2"])

,Customer ID,Food ID - WK1,Food ID - WK2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


In [16]:
# joining on multiple columns
week1.merge(week2, how="inner", on=["Customer ID", "Food ID"])

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


## full outer join

In [23]:
merged = week1.merge(week2, how="outer", on="Customer ID", suffixes=[" - WK1", " - WK2"], indicator=True)
# NaN value indicates that the value didn't exist
# 62 common Customer IDs between the two datasets 
merged

,Customer ID,Food ID - WK1,Food ID - WK2,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


In [25]:
merged["_merge"].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

In [26]:
# filter for those rows that are present in either Week1 or Week2, but not both
merged[merged["_merge"] != "both"]

,Customer ID,Food ID - WK1,Food ID - WK2,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


## left join

In [35]:
week1.merge(foods, how="left", on = "Food ID", sort=True)

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
...,...,...,...,...
245,809,10,Drink,1.75
246,584,10,Drink,1.75
247,274,10,Drink,1.75
248,151,10,Drink,1.75


In [36]:
# get only particular columns
week1.merge(foods, how="left", on = "Food ID", sort=True)[["Customer ID", "Food Item", "Price"]]

,Customer ID,Food Item,Price
0,658,Sushi,3.99
1,600,Sushi,3.99
2,155,Sushi,3.99
3,341,Sushi,3.99
4,20,Sushi,3.99
...,...,...,...
245,809,Drink,1.75
246,584,Drink,1.75
247,274,Drink,1.75
248,151,Drink,1.75


## merging with different column names

In [38]:
week2.merge(customers, how="left", left_on="Customer ID", right_on="ID", sort=True)

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,8,Frances,Adams,Female,Dabshots,Developer III
1,13,2,13,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,21,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,24,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,27,Jessica,Bennett,Female,Twitternation,Account Executive
...,...,...,...,...,...,...,...,...
245,968,4,968,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV
246,969,3,969,Marie,Ortiz,Female,Meejo,Research Associate
247,977,7,977,Cynthia,Dixon,Female,Skalith,Automation Specialist II
248,994,2,994,Clarence,Morgan,Male,Edgewire,Geologist II


## merge on index

In [39]:
# merget between Cusomter ID of week1 and index of customers
week1.merge(customers, how="left", left_on="Customer ID", right_index=True)

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,538.0,Sara,Washington,Female,Skimia,Database Administrator III
1,97,4,98.0,Jean,Mills,Female,Fivebridge,Product Engineer
2,658,1,659.0,Howard,Hudson,Male,Topicshots,Professor
3,202,2,203.0,Annie,Lane,Female,Skyble,Marketing Manager
4,155,9,156.0,Stephanie,Kelly,Female,Youopia,Automation Specialist III
...,...,...,...,...,...,...,...,...
245,413,9,414.0,Harold,Adams,Male,Dazzlesphere,Account Coordinator
246,926,6,927.0,Nicholas,Morris,Male,Linkbridge,Assistant Professor
247,134,3,135.0,Richard,Murphy,Male,Cogidoo,Software Consultant
248,396,6,397.0,Brenda,Ryan,Female,Oozz,Nurse Practicioner


## join method

In [40]:
satisfaction = pd.read_csv("data/Restaurant - Week 1 Satisfaction.csv")
satisfaction

,Satisfaction Rating
0,2
1,7
2,3
3,7
4,10
...,...
245,1
246,2
247,8
248,10


In [41]:
# user merge method to join on index
week1.merge(satisfaction, how = "left", left_index=True, right_index=True)

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
...,...,...,...
245,413,9,1
246,926,6,2
247,134,3,8
248,396,6,10


In [42]:
# will automatically join on index
week1.join(satisfaction)

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
...,...,...,...
245,413,9,1
246,926,6,2
247,134,3,8
248,396,6,10


## pd.merge() almost same as dataframe merge

In [45]:
pd.merge(week1, customers, how="left", left_on="Customer ID", right_on="ID")

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,537,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,97,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,658,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,202,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,155,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...,...
245,413,9,413,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,926,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,134,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,396,Juan,Romero,Male,Zoonder,Analyst Programmer
